# Image Scrapper Roughwork

# Non OOP apprach

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [3]:
import os
import time

In [28]:
# enterign search_criteria
search_criteria = input("enter the search item: ").replace(' ','')
print(search_criteria)

enter the search item: Birla
Birla


In [29]:
#defining driver
DRIVER_PATH = './chromedriver'
def scroll_to_end(wd):
#load the fill page instead of scroll 
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#defining the sleep between each hit at the server    
        time.sleep(sleep_between_interactions)
#building the url to search    
search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img".format(q=search_criteria) 
search_url

'https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q=Birla&oq=Birla&gs_l=img'

In [30]:
#initializing the driver
driver = webdriver.Chrome()

In [31]:
#loading the page in the chrome driver that was initialized before
driver.get(search_url)

In [32]:
#getting the details of each image thumbnails in a list - all post
all_posts = driver.find_elements_by_class_name("Q4LuWd")
all_posts

[<selenium.webdriver.remote.webelement.WebElement (session="e63b650a9967c3c3818e7acbd0588244", element="0bb1f88a-6588-4609-9c26-cd4aba9212da")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e63b650a9967c3c3818e7acbd0588244", element="9d77615c-98f8-4f47-ab7a-9931b339cfa6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e63b650a9967c3c3818e7acbd0588244", element="9355ee38-2c9b-4507-b9e5-2f455eb92c4c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e63b650a9967c3c3818e7acbd0588244", element="6ff7fb72-87ca-4554-8ead-f6e9e2c8e366")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e63b650a9967c3c3818e7acbd0588244", element="7e4d5963-4178-4b9b-b11b-b34b64f779eb")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e63b650a9967c3c3818e7acbd0588244", element="2635052a-98e4-44a4-9ab4-a369d8c08dd4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e63b650a9967c3c3818e7acbd0588244", element="ce8a0a21-a7e9-4c68-af66-bf

In [33]:
#initializing an enpty list to append urls
image_url = []
# instead of reading all post which would be service heavy - we are reading only 15 such thumbnails and looping through them
for post in all_posts[0:15]:
    try:
#imitating browser action to click each thumbnail        
        post.click()
#also defining timesleep bwtn clicks    
        time.sleep(0.5)
    except Exception:
        continue
#after clicking a separate page opens up with the image source, so extracting the entire 'n3VNCb' block to filter the src url from it
#driver.find_elements_by_css_selector('img:n3VNCb') could also have been used
    actual_images = driver.find_elements_by_class_name('n3VNCb')
    for actual_image in actual_images:
#checking to make sure each of this extracted links have src and http in it        
          if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                image_url.append(actual_image.get_attribute('src'))

In [34]:
#printing the list of urls appended in image_url
image_url

['https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Lalu_Prasad_presenting_the_JRD_Tata_Corporate_Leadership_Award_to_Shri_Kumar_Mangalam_Birla%2C_Chairman%2C_Aditya_Birla_Group%2C_Mumbai%2C_at_the_Foundation_Day_of_All_India_Management_Association_%28AIMA%29%2C_in_New_Delhi.jpg/1200px-thumbnail.jpg',
 'https://images.financialexpress.com/2019/07/birla.jpg']

In [35]:
len(image_url)

2

In [37]:
#we want to save the image as search_critera_1.jpg such as apple_1.jpg, tree_2.jpg in the target_image folder
#1,2 in the file name is set by the counter
counter = 0
target_path='./target_image'
#looping through each url in the image_url lust
for image in image_url:
#extracting the image details of pixels in image_content    
    image_content = requests.get(image).content
#writing the binary file in /jpg    
    f = open(os.path.join(target_path, f"{search_criteria}" + "_" + str(counter) + ".jpg"), 'wb')
    f.write(image_content)
    f.close()
    counter = counter+1

In [38]:
#getting the list of all the files_names so we can delete them in future
list_of_jpg_files=[]
list_of_files=os.listdir(target_path)
list_of_files

['Birla_0.jpg', 'Birla_1.jpg']

In [39]:
#deleting the files before new search critera
for i in list_of_files:
    try:
        os.remove(f"./{target_path}/"+ i)
    except Exception as e:
        print('error in deleting:  ',e)
    

# OOPS Approach

In [15]:
class imageScrapperUtilities:
    def __init__(self, target_path):
        self.target_path = target_path
        
    def emptyTargetDir(self):
        list_of_jpg_files=[]
        list_of_files=os.listdir(self.target_path)
        if len(list_of_files)>0:
            for i in list_of_files:
                    try:
                        os.remove(f"./{self.target_path}/"+ i)
                    except Exception as e:
                        print('error in deleting:  ',e)  
    
    
    
            
                          

In [16]:
class fetchAndSaveImage:
    def __init__(self, sleep_between_interactions,target_path,search_criteria,driver_path):
        self.sleep_between_interactions = sleep_between_interactions
        self.driver_path = driver_path
        self.target_path = target_path
        self.search_criteria = search_criteria
        self.driver = webdriver.Chrome(executable_path=self.driver_path)   
        self.search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img".format(q=search_criteria) 
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
              
        print(self.search_url)   
    
    
    def gatherImageSrc(self):
        self.driver.get(self.search_url)
        all_posts = self.driver.find_elements_by_class_name("Q4LuWd")
        print(len(all_posts))
        self.image_url_list = []
        for post in all_posts[0:20]:
            try:
                post.click()
                time.sleep(sleep_between_interactions)
        
            except Exception:
                continue
            print('----------------')
            actual_images = self.driver.find_elements_by_class_name('n3VNCb')
            for actual_image in actual_images:
                  if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                           self.image_url_list.append(actual_image.get_attribute('src'))
        print(self.image_url_list ) 
        print('$$$$$$$$$$$')
        print(len(self.image_url_list) ) 
        return self.image_url_list  
    
    def saveImage(self):
        counter = 0
        for image in self.image_url_list:
            image_content = requests.get(image).content
            f = open(os.path.join(self.target_path, f"{self.search_criteria}" + "_" + str(counter) + ".jpg"), 'wb')
            f.write(image_content)
            f.close()
            counter = counter+1

In [17]:
driver_path = './chromedriver'
sleep_between_interactions = 0.5
target_path='./target_image'
search_criteria = input("enter the search item: ").replace(' ','')
print(search_criteria)

enter the search item: tata
tata


In [18]:
def search_and_download(target_path,sleep_between_interactions,search_criteria,driver_path):
        
        
        
        #with webdriver.Chrome(executable_path=driver_path) as driver:
            #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            imageScrapperUtilitiesObject = imageScrapperUtilities(target_path)
            fetchAndSaveImageObject = fetchAndSaveImage(sleep_between_interactions,target_path,search_criteria,driver_path)
            imageScrapperUtilitiesObject.emptyTargetDir()    
            fetchAndSaveImageObject.gatherImageSrc()
            fetchAndSaveImageObject.saveImage()
        
        

In [19]:
search_and_download(target_path,sleep_between_interactions,search_criteria,driver_path)    

https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q=tata&oq=tata&gs_l=img
48
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
----------------
['https://etimg.etb2bimg.com/photo/73698519.cms', 'https://images.carandbike.com/car-images/colors/tata/nexon/tata-nexon-calgary-white.webp?v=1579759339', 'https://images.moneycontrol.com/static-mcnews/2019/12/Nexon-EV-5-770x433.jpg?impolicy=website&width=770&height=431', 'https://www.ft.com/__origami/service/image/v2/images/raw/https%3A%2F%2Fs3-ap-northeast-1.amazonaws.com%2Fpsh-ex-ftnikkei-3937bb4%2Fimages%2F1%2F6%2F4%2F2%2F19322461-4-eng-GB%2FCropped-1550000211%E3%82%BF%E3%82%BF%E3%81%AE%E3%81%97%E3%81%8B%E3%81%91%EF%BC%B0%EF%BC%9A%E3%82%BF%E3%82%BF%E8%87%AA%E